In [3]:
import pandas as pd
import nltk #Natural languange Processing tool
nltk.download('punkt')
import json
from nltk.stem.lancaster import LancasterStemmer #model for stemming words
stemmer = LancasterStemmer()
# df = pd.read_excel('/content/Omdena Chatbot Dataset.xlsx')
# df.head(10)

[nltk_data] Downloading package punkt to /home/loise/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
ourData = json.loads(open('resources/job_intents.json').read())
ourData

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hi there',
    'Hey',
    'hey',
    'hi',
    'hi there',
    'how are you',
    'How are you',
    'sasa',
    'SaSa',
    'sema',
    'Sema',
    'Habari',
    'Habari yako',
    'uko aje',
    'Ukoje',
    'hello',
    'Hello',
    'Good evening',
    'Good morning',
    'Good afternoon',
    'Morning',
    'evening',
    'afternoon',
    'hello there',
    'Hello there',
    'mambo',
    'Mambo'],
   'responses': ['Hi stranger',
    'poa',
    'Hi too',
    'hi too',
    'hello',
    'Hola',
    'hello, how can I help you?',
    'Poa',
    'Mzuri',
    'mzuri']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Ok bye',
    'Bye Bye',
    'bye',
    'bye bye',
    'okay',
    'Okay',
    'wakati mwema'],
   'responses': ['See you!',
    'Have a nice day',
    'shinda poa',
    'sure bye',
    'Sure byeeeee',
    'Same to you']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank y

In [5]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")
nltk.download("wordnet")

2022-09-06 15:49:08.766042: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-06 15:49:08.775005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 15:49:08.775038: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/loise/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/loise/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/loise/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
def _lowercase(obj):
    """ Make dictionary lowercase """
    if isinstance(obj, dict):
        return {k.lower():_lowercase(v) for k, v in obj.items()}
    elif isinstance(obj, (list, set, tuple)):
        t = type(obj)
        return t(_lowercase(o) for o in obj)
    elif isinstance(obj, str):
        return obj.lower()
    else:
        return obj.translate(str.maketrans('', '', string.punctuation))
ourData = _lowercase(ourData)
import pandas as pd
import nltk #Natural languange Processing tool
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer #model for stemming words
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/loise/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


###Creating character/word mappings

In [8]:
import numpy as np
from keras.utils import np_utils
for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        characters = sorted(list(set(pattern)))
        n_to_char = {n:char for n, char in enumerate(characters)}
        char_to_n = {char:n for n, char in enumerate(characters)}

        X = []
        Y = []
        length = len(pattern)
        seq_length = 100
        for i in range(0, length-seq_length, 1):
            sequence = ourData[i:i + seq_length]
            label =ourData[i + seq_length]
            X.append([char_to_n[char] for char in sequence])
            Y.append(char_to_n[label])

            X_modified = np.reshape(X, (len(X), seq_length, 1))
            X_modified = X_modified / float(len(characters))
            Y_modified = np_utils.to_categorical(Y)


In [9]:
lm = WordNetLemmatizer() 
ourClasses = []
newWords = []
documentX = []
documentY = []

for intent in ourData["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] 
newWords = sorted(set(newWords))
ourClasses = sorted(set(ourClasses))


In [10]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

In [21]:
# defining some parameters
iShape = (len(x[0]),)
oShape = len(y[0])

# the deep learning model
ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(Dropout(0.5))
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tensorF.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
hist = ourNewModel.fit(x, y, epochs=200, verbose=1)
ourNewModel.save('chatbot_model.h5', hist)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               13312     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 15)                975       
                                                                 
Total params: 22,543
Trainable params: 22,543
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
5/5 [============================

In [14]:
!pip install autocorrect
from autocorrect import Speller
import string
def ourText(text):
  spell = Speller()
  newtkns = nltk.word_tokenize(spell(text.translate(str.maketrans('', '', string.punctuation))))
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(ourResult)



Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install SpeechRecognition numpy gTTs sklearn


In [ ]:
import pyttsx3
engine = pyttsx3.init()
# convert this text to speech
text = "Python is a great programming language"
engine.say(text)
# play the speech
engine.runAndWait()

In [ ]:
def AutoCorrect(text):
  spell = Speller()
  correct = spell(text.translate(str.maketrans('', '', string.punctuation)))
  return correct

print(AutoCorrect("whay is rhhis"))

In [ ]:
!sudo apt update && sudo apt install espeak ffmpeg libespeak1

In [ ]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio #Import Audio method from IPython's Display Class
tts = gTTS("lets dekut meet for the weekly meeting update") #Provide the string to convert to speech
tts.save('1.wav') #save the string converted to speech as a .wav file
sound_file = '1.wav'
Audio(sound_file, autoplay=True) 

In [ ]:
!pip install transformers sentencepiece sacremoses

In [ ]:
from transformers import *
# import transformers
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

sentence = "Kindly elaborate your issue."
get_paraphrased_sentences(model, tokenizer, sentence, num_beams=3, num_return_sequences=3)

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob
res = TextBlob("tell me about dekut")
print(res.sentiment.polarity)

In [12]:
import pickle

In [17]:
outfile = "ourModel.sav"
with open(outfile, 'wb') as files:
    pickle.dump(ourNewModel, files)
outfile.close()


INFO:tensorflow:Assets written to: ram://81eb1000-42b4-4562-894e-03a2a74a2d6e/assets


AttributeError: 'str' object has no attribute 'close'

In [14]:
with open('classes.pkl', 'wb') as files:
    pickle.dump(ourClasses, files)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
for intent in ourData['intents']:
   for pattern in intent['patterns']:
   # take each word and tokenize it
w = nltk.word_tokenize(pattern)
words.extend(w)
# adding documents
documents.append((w, intent['tag']))

# adding classes to our class list
if intent['tag'] not in classes:
   classes.append(intent['tag'])

In [20]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
intents = json.loads(open('resources/job_intents.json').read())

for intent in intents['intents']:
   for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))
        # adding classes to our class list
if intent['tag'] not in classes:
    classes.append(intent['tag'])

    words = [lemmatizer.lemmatize(w.lower()) for w in words
    if w not in ignore_words
    ]
    words = sorted(list(set(words)))

    classes = sorted(list(set(classes)))

    print(len(documents), "documents")

    print(len(classes), "classes", classes)

    print(len(words), "unique lemmatized words", words)

    pickle.dump(words, open('resources/words.pkl', 'wb'))
    pickle.dump(classes, open('resources/classes.pkl', 'wb'))



133 documents
1 classes ['club']
105 unique lemmatized words [',', '.', '...', 'a', 'about', 'afternoon', 'aje', 'am', 'any', 'are', 'asanti', 'at', 'awesome', 'bye', 'called', 'can', 'certificate', 'city', 'club', 'course', 'dedan', 'degree', 'dekut', 'diploma', 'employed', 'employment', 'evening', 'explain', 'find', 'for', 'found', 'general', 'get', 'give', 'good', 'goodbye', 'habari', 'hello', 'help', 'helpful', 'here', 'hey', 'hi', 'how', 'i', 'in', 'information', 'is', 'job', 'kimathi', 'later', 'list', 'located', 'location', 'mambo', 'many', 'master', 'me', 'morning', 'much', 'mwema', 'my', 'name', 'nashukuru', 'of', 'offered', 'offerred', 'ok', 'okay', 'opportunites', 'opportunity', 'phd', 'please', 'postgraduate', 'programme', 'sasa', 'see', 'sema', 'so', 'taught', 'technology', 'techology', 'tell', 'tender', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'uko', 'ukoje', 'undergradaute', 'undergraduate', 'universitiy', 'university', 'wa', 'wakati', 'want', 'what', 'where', 'w

[nltk_data] Downloading package punkt to /home/loise/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/loise/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
      # initializing bag of words
   bag = []

   pattern_words = doc[0]
   # lemmatize each word - create base word, in attempt to represent related words
   pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
   # create our bag of words array with 1,
   for w in words:
      if w in pattern_words:
         bag.append(1) 

      else:
        bag.append(0)

   # output is a '0'

   output_row = list(output_empty)
   output_row[classes.index(doc[1])] = 1

   training.append([bag, output_row])
   # shuffle our features and turn into np.array
   random.shuffle(training)
   training = np.array(training)
   # create train and test lists.X - patterns, Y - intents
   train_x = list(training[: , 0])
   train_y = list(training[: , 1])
   print("Training data created")


   # Create model - 3 layers.First layer 128 neurons, second layer 64 neurons and 3 rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]), ), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))


sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot_model.h5', hist)

print("model created")

ValueError: 'greeting' is not in list